In [ ]:
#import dependencies
import pandas as pd
import numpy as np

In [ ]:
#read file
beers_pd = pd.read_csv("resources/open-beer-database.csv")
beers_pd.head()

In [ ]:
#select columns
beers_df = beers_pd[['Name','Alcohol By Volume','Style','Category','Brewer','City','State','Country']]
beers_df.head()

In [ ]:
#removed NaNs from Style
beers_df = beers_df[beers_df['Style'].notna()]
beers_df.head()


In [ ]:
# remove US from Country
beers_no_us = beers_df.loc[beers_df.Country.values != 'United States']
beers_no_us.head()

In [ ]:
#change column names
beers_no_us = beers_no_us.rename(columns={'Alcohol By Volume': 'abv','Name':'Beer Name','Brewer':'Brewery Name'})
beers_no_us

In [ ]:
#sort breweries for count off
beers_no_us = beers_no_us.sort_values(by=['Brewery Name'], ascending=[True])
beers_no_us

In [ ]:
#group breweries to establish a count and prepare to add an id # that will work with our us data
grouped_brewery = beers_no_us.groupby(['Brewery Name']).count()
grouped_brewery


In [ ]:
#create Brewery ID function
brew_id = 577
brewery_id = []
for row in range(len(grouped_brewery['Country'])):
    brew_id = brew_id + 1
    brewery_id.append(brew_id)
    


    


grouped_brewery['Brewery Id']=brewery_id
grouped_brewery

In [ ]:
#merge DataFrames to make it easier to pull the values we need for each table.
full_df = beers_no_us.merge(grouped_brewery, left_on = ['Brewery Name'], right_on = ['Brewery Name'], how = 'left')
full_df

In [ ]:
#create our international brewery dataframe 
international_brewery = full_df[['Brewery Id', 'Brewery Name', 'City_x', 'State_x', 'Country_x', 'City_y']]
international_brewery

In [ ]:
#drop duplicate rows and rename the columns
international_brewery = international_brewery.drop_duplicates(['Brewery Name'], keep = 'first')
international_brewery = international_brewery.rename(columns = {'City_x':'city', 'State_x':'state', 'Country_x':'country', 'City_y':'beer_count', 'Brewery Id':'brewery_id','Brewery Name':'brewery_name'})
international_brewery

In [ ]:
#import items to run api requests from google
import requests
import json
from config import gkey


In [ ]:
#establish a base Url and a list of values to search
base_url = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input='
brewery= international_brewery['brewery_name']

In [ ]:
#create a value to return and runa for loop the request the rating of each brewery from google.
rating = []
for b in brewery:
    query_url = f'{base_url}{b}&inputtype=textquery&fields=name,rating&key={gkey}'
    response = requests.get(query_url)
    response_json = response.json()
    try:
      rating.append(response_json['candidates'][-1]['rating'])
    except:
        rating.append('NaN')
        
rating  

In [ ]:
#add the rating to the DataFrame
international_brewery['rating'] = rating
international_brewery = international_brewery.drop(columns = ['Rating'])
international_brewery

In [ ]:
#create our international beers DataFrame and rename for consistency
international_beers = full_df[['Beer Name_x', 'Style_x','abv_x', 'Brewery Id']]
international_beers = international_beers.rename(columns = {'Beer Name_x':'beer_name', 'Style_x':'style', 'abv_x':'ABV', 'Beer Name':'beer_name','Brewery Id':'brewery_id'})
international_beers

In [ ]:
#save as a csv so we don't have to run the API code again to load our DataFrames into SQL.
international_brewery.to_csv('output/final_international_brewery.csv')
international_beers.to_csv('output/final_international_beers.csv')


In [ ]:
#import sqlalchemy
from sqlalchemy import create_engine

In [ ]:
#establish connection to the database
connection_string = "postgres:postgres@localhost:5432/Beer_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
#check table have been created
engine.table_names()

In [ ]:
#push the beers dataframe to sql
international_beers.to_sql(name = 'beer_tbl', con = engine, if_exists='append', index = False)

In [ ]:
#push the brewery dataframe to sql
international_brewery.to_sql(name = 'brewery_tbl', con = engine, if_exists='append', index = False)